In [ ]:
import pandas as pd
import re
%pip install contractions
import contractions
import numpy as np
np.random.seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
data = pd.read_csv('data/train.csv')

In [ ]:
class Text:

    def remove_numbers_and_non_english(self, text):

        text = re.sub(r'\d+', '', text)

        # Remove non-English characters (assuming English alphabet)
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        return text

    def expand_contractions(self, text):
        expanded_text = contractions.fix(text)

        return expanded_text

    def remove_extraSpace(self, text):
        text = re.sub(' +', ' ', text)
        # Remove trailing and leading whitespaces in string
        text = text.strip()
        return text

    def remove_newline(self, text):
        text = text.replace("\n", " ")
        return text

    @staticmethod
    def preprocess(text):
        instance = Text()
        text = instance.remove_newline(text)
        text = instance.expand_contractions(text)
        text = instance.remove_numbers_and_non_english(text)
        text = instance.remove_extraSpace(text)
        return text

In [ ]:
data['comment_text'] = data['comment_text'].apply(Text.preprocess)

In [ ]:
X = data['comment_text']

In [ ]:
y = data.iloc[:, 2:]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y ,
                                   random_state=42,
                                   test_size=0.2,
                                   shuffle=True)

In [ ]:
vocab = set()
for sentence in X_train:
    for word in sentence.split():
        vocab.add(word)

print(len(vocab))

232534


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
tokenizer

In [ ]:
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_val_tokenized = tokenizer.texts_to_sequences(X_val)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_sequences = pad_sequences(X_train_tokenized, maxlen=300,
                                padding="post", truncating="post")
val_sequences = pad_sequences(X_val_tokenized, maxlen=300,
                              padding="post", truncating="post")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, TextVectorization, Input

In [ ]:
model = tf.keras.Sequential([
    Embedding(25000, 200, input_length=300),
    LSTM(16, activation="tanh"),
    Dense(6, activation="sigmoid"), # we are predicting for 6 classes
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 200)          5000000   
                                                                 
 lstm (LSTM)                 (None, 16)                13888     
                                                                 
 dense (Dense)               (None, 6)                 102       
                                                                 
Total params: 5013990 (19.13 MB)
Trainable params: 5013990 (19.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)


In [ ]:
epochs = 5

In [ ]:
batch_size = 64
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
]

# model.evaluate(X_val, y_val)

model.fit(
    train_sequences,
    y_train,
    validation_data=(val_sequences, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
)

Epoch 1/5
1995/1995 [==============================] - 175s 84ms/step - loss: 0.1500 - accuracy: 0.9457 - val_loss: 0.1408 - val_accuracy: 0.9939
Epoch 2/5
1995/1995 [==============================] - 50s 25ms/step - loss: 0.1398 - accuracy: 0.9938 - val_loss: 0.1406 - val_accuracy: 0.9940
Epoch 3/5
1995/1995 [==============================] - 41s 20ms/step - loss: 0.1353 - accuracy: 0.9930 - val_loss: 0.1403 - val_accuracy: 0.9940
Epoch 4/5
1995/1995 [==============================] - 37s 19ms/step - loss: 0.1369 - accuracy: 0.9941 - val_loss: 0.0976 - val_accuracy: 0.9938
Epoch 5/5
1995/1995 [==============================] - 35s 17ms/step - loss: 0.0667 - accuracy: 0.9898 - val_loss: 0.0573 - val_accuracy: 0.9941


In [ ]:
model.save_weights('weights.h5')
print('Model Saved!')

Model Saved!


In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)